In [1]:
#!pip uninstall Keras -y ; pip install Keras==2.3.0
#!conda install -c bioconda pybedtools -y 
#!conda install -c bioconda pybigwig -y 
#!conda install -c anaconda tensorflow==1.15.0 -y 
#!pip install google
#!pip install protobuf
#!pip install tensorflow-gpu

In [2]:
from keras.models import load_model
import numpy as np
import pyBigWig
import pybedtools

Using TensorFlow backend.


In [13]:
model_file = '../Merged_100_filters_conservation_all_strands_merged_model_first_test-40-0.71.hdf5'

In [14]:
model_merged = load_model(model_file)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [15]:
base_encode = {
    'A': [0, 0, 0, 1],
    'C': [0, 0, 1, 0],
    'T': [0, 1, 0, 0],
    'G': [1, 0, 0, 0]
}

In [16]:
!pwd

/srv/notebooks/microexon_predictor


In [25]:
!wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/phastCons100way/hg19.100way.phastCons.bw

--2020-02-28 17:26:47--  http://hgdownload.cse.ucsc.edu/goldenpath/hg19/phastCons100way/hg19.100way.phastCons.bw
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777277074 (5.4G) [text/plain]
Saving to: 'hg19.100way.phastCons.bw'

hg19.100way.phastCo 100%[===================>]   5.38G  9.51MB/s    in 7m 0s   

2020-02-28 17:33:47 (13.1 MB/s) - 'hg19.100way.phastCons.bw' saved [5777277074/5777277074]



In [26]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
!gunzip -d hg19.fa.gz

--2020-02-28 17:33:48--  https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948731419 (905M) [application/x-gzip]
Saving to: 'hg19.fa.gz'

hg19.fa.gz          100%[===================>] 904.78M  10.9MB/s    in 27s     

2020-02-28 17:34:15 (33.2 MB/s) - 'hg19.fa.gz' saved [948731419/948731419]



In [19]:
GENOME=  '/srv/notebooks/microexon_predictor/hg19.fa' 
CONSERVATION_FILE = '/srv/notebooks/microexon_predictor/hg19.100way.phastCons.bw' 




In [20]:
def capture_values_bw(big_wig, 
                      chr,
                      start,
                      end):
    try:
        with pyBigWig.open(big_wig) as bw:
            values_vector  = bw.values(chr, start, end)
            return values_vector
    except Exception as e:
        print (chr,start,end)
        print (e)
        

In [21]:
def capture_fasta_sequences(fasta_file, 
                      chr,
                      start,
                      end):



    fasta_query = pybedtools.BedTool("{} {} {}".format(chr, start, end), from_string=True)
    fasta_query_out = fasta_query.seq((chr, start, end), fasta_file)
    return fasta_query_out

In [22]:
#capture_fasta_sequences(GENOME,'chr1', 3000000, 3000020 )



In [23]:
class Microexon():
    def __init__(self, chr, start, end, strand):
        self.chr  = chr
        self.start  = start
        self.end  = end
        self.strand  = strand
        self.flank_up_limit_start = self.start - 1
        self.flank_up_limit_end = self.start - 101
        self.flank_down_limit_start = self.end + 1
        self.flank_down_limit_end = self.end + 101
        self.conservation_values_up = None
        self.conservation_values_down = None
        self.seq_values_up = None
        self.seq_values_down = None
        self.set_values()
        self.encode_dna_up = self.encoding_dna_sequence(self.seq_values_up)
        self.encode_dna_down = self.encoding_dna_sequence(self.seq_values_down) 
        
        

        
    def set_values(self):
        '''

        '''
        self.conservation_values_up = np.array(capture_values_bw(CONSERVATION_FILE,
                          self.chr,
                          self.flank_up_limit_end,
                          self.flank_up_limit_start)).reshape(1,100,1)
        
        self.conservation_values_down = np.array(capture_values_bw(CONSERVATION_FILE,
                  self.chr,
                  self.flank_down_limit_start,
                  self.flank_down_limit_end)).reshape(1,100,1)

        
        self.seq_values_up = capture_fasta_sequences(GENOME,
                          self.chr,
                          self.flank_up_limit_end,
                          self.flank_up_limit_start)
        
        self.seq_values_down = capture_fasta_sequences(GENOME,
                  self.chr,
                  self.flank_down_limit_start,
                  self.flank_down_limit_end)
        
        
#         print (map(len, [self.conservation_values_up,
#                          self.conservation_values_down,
#                         self.seq_values_up,
#                         self.seq_values_down,]))
        
    def encoding_dna_sequence(self, x):
        return np.array([base_encode[x_nucleo.upper()] for x_nucleo in x ])
    
    
    
    def prediction(self):
        '''I need to flip strands at this point'''
        if self.strand == '+':
            out_prob = model_merged.predict([self.encode_dna_up.reshape(1, 100, 4),
                     self.conservation_values_up.reshape(1,100,1),
                     self.encode_dna_down.reshape(1, 100, 4),
                     self.conservation_values_down.reshape(1,100,1)] )
            
            
        if self.strand == '-':
            out_prob = model_merged.predict([self.encode_dna_down[::-1].reshape(1, 100, 4),
                     self.conservation_values_down[::-1].reshape(1, 100, 1),
                     self.encode_dna_up[::-1].reshape(1, 100, 4),
                     self.conservation_values_up[::-1].reshape(1,100,1)] )
        
        print (out_prob[0][0])
        
        
        

In [24]:
m1 = Microexon('chr1', 3000000, 3000020, '-')
m1.prediction()

0.6618371
